In [1]:
%cd /content/drive/My Drive/Colab Notebooks/

[Errno 2] No such file or directory: '/content/drive/My Drive/Colab Notebooks/'
/Users/ajinkyakulkarni/Desktop/kaggle_task


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm 
import cv2
from keras.utils.np_utils import to_categorical

2023-03-16 15:12:51.372937: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.

KeyboardInterrupt



In [ ]:
data_dir = '/content/drive/My Drive/Colab Notebooks/SamplesDebasis'
categories = os.listdir(data_dir)
print(categories)

In [ ]:
for category in categories:
  category_path = os.path.join(data_dir, category)
  samples = os.listdir(category_path)
  for sample in samples:
    img = cv2.imread(os.path.join(category_path, sample))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (128,128))
    img = img/255
    plt.imshow(img)
    plt.title(category)
    plt.show()
    break

In [ ]:
data = []
label = [] 
for category in categories:
  category_path = os.path.join(data_dir, category)
  samples = os.listdir(category_path)
  for sample in tqdm(samples):
    img = cv2.imread(os.path.join(category_path, sample))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (128,128))
    img = img/255

    data.append(img)
    label.append(categories.index(category)) 

data = np.asarray(data)
label = np.asarray(label)
label = to_categorical(label, num_classes=4)



In [ ]:
from sklearn.utils import shuffle

In [ ]:
X_train, y_train = shuffle(data, label, random_state=3)

In [ ]:
# specify version of tensorflow
#%tensorflow_version 1.x
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(8, (3, 3), padding='same', activation='relu', input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))


# FC layer
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(rate = 0.3))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(rate = 0.3))
model.add(tf.keras.layers.Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
model.summary()

In [ ]:
hist = model.fit(X_train, y_train, batch_size=16,
                        epochs=50, verbose=1,
                        validation_split=0.2)

In [ ]:
# Display models statistics
loss = hist.history['loss']
val_loss = hist.history['val_loss']
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
epochsr = range(50)
plt.figure()
plt.plot(epochsr, loss, 'bo', label='Training loss')
plt.plot(epochsr, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()
plt.figure()
plt.plot(epochsr, acc, 'bo', label='Training acc')
plt.plot(epochsr, val_acc, 'b', label='Validation acc')
plt.title('Accuracy')
plt.legend()
plt.show()
mymodel = hist.model